<a href="https://colab.research.google.com/github/harshit4311/backtrader-library-test/blob/main/backtrader_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
print('hello')

hello


## About the "backtrader" lib:

- the "backtesting" framework is object-oriented
- expects you to define trading strats as a "Class" that inherits from "strategy"

#### Why this class-based design makes sense:

1. Encapsulation
- all your strategy logic (init, next, indicators, position handling, etc.) inside one class -- clean & modular


2. Reusability
- you can easily reuse and extend strats.
- Let's say you want to try the same logic but with a trailing stop -- Inherit from "TrailingStrategy" class


3. Framework integration
backtester needs to:
- instatiate your strategy
- call your "next()" function bar-by-bar
- access your indicators and logic

#### For more details/all functions, checkout the documentation:
https://kernc.github.io/backtesting.py/doc/backtesting/lib.html#gsc.tab=0

In [128]:
!pip install backtrader
!pip install plotly

In [129]:
import pandas as pd
import backtrader as bt
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [130]:
df = pd.read_csv('/content/BTC1H.csv')

In [131]:
df.head()

,datetime,open,high,low,close,volume
0,2011-12-31 07:00:00,4.39,4.39,4.39,4.39,2.277904
1,2011-12-31 08:00:00,4.39,4.39,4.39,4.39,27.334852
2,2011-12-31 09:00:00,4.39,4.39,4.39,4.39,27.334852
3,2011-12-31 10:00:00,4.39,4.39,4.39,4.39,27.334852
4,2011-12-31 11:00:00,4.39,4.39,4.39,4.39,27.334852


In [132]:
df.shape

(80426, 6)

In [133]:
df.columns

Index(['datetime', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

In [134]:
import plotly.graph_objects as go

# Just to be safe: make sure datetime is datetime dtype
df['datetime'] = pd.to_datetime(df['datetime'])

# Create candlestick chart
fig = go.Figure(data=[go.Candlestick(
    x=df['datetime'],
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close']
)])

fig.update_layout(
    title='BTC Candlestick Chart',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    xaxis_rangeslider_visible=False
)

fig.show()


In [135]:
import backtrader as bt

# Ensure your 'datetime' column is the index (Backtrader expects the datetime column to be in index)
df.set_index('datetime', inplace=True)

# Convert your DataFrame to Backtrader’s data feed
class PandasData(bt.feeds.PandasData):
    lines = ('openinterest',)
    params = (
        ('datetime', None),
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('openinterest', -1)
    )

# Create the data feed
data = PandasData(dataname=df)

In [136]:
import backtrader as bt

class MovingAverageCrossover(bt.Strategy):
    # Initialize the strategy and indicators inside the __init__ method
    def __init__(self):
        # Define the moving averages using the 'close' price (self.data.close)
        self.short_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=10)
        self.long_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=30)

        # Define crossover indicator
        self.cross_over = bt.indicators.CrossOver(self.short_ma, self.long_ma)

    def next(self):
        # Buy signal: Short MA crosses above Long MA
        if self.cross_over > 0:
            if not self.position:  # Ensure no position is open
                self.buy()  # Enter a buy order

        # Sell signal: Short MA crosses below Long MA
        elif self.cross_over < 0:
            if self.position:  # If there's an open position
                self.sell()  # Exit the position


In [137]:
# Install backtrader (if not already installed)
!pip install backtrader

import backtrader as bt
import pandas as pd

# Load the data
df = pd.read_csv('/content/BTC1H.csv')

# Convert the datetime column to datetime format
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

# Create a Backtrader data feed from the DataFrame
data = bt.feeds.PandasData(dataname=df)

# Define the Moving Average Crossover Strategy
class MovingAverageCrossover(bt.Strategy):
    def __init__(self):
        # Define the moving averages using the 'close' price
        self.short_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=10)
        self.long_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=30)

        # Define crossover indicator
        self.cross_over = bt.indicators.CrossOver(self.short_ma, self.long_ma)

    def next(self):
        # Buy signal: Short MA crosses above Long MA
        if self.cross_over > 0:
            if not self.position:  # Ensure no position is open
                self.buy()  # Enter a buy order
                print('Buy signal at', self.data.datetime.datetime(), 'Price:', self.data.close[0])

        # Sell signal: Short MA crosses below Long MA
        elif self.cross_over < 0:
            if self.position:  # If there's an open position
                self.sell()  # Exit the position
                print('Sell signal at', self.data.datetime.datetime(), 'Price:', self.data.close[0])

# Initialize the backtest engine (Cerebro)
cerebro = bt.Cerebro()

# Add the data to the engine
cerebro.adddata(data)

# Set the initial cash (capital) for the backtest
cerebro.broker.set_cash(10000)

# Set the commission for each trade (0.1%)
cerebro.broker.setcommission(commission=0.001)

# Set the position size (1 BTC per trade)
cerebro.addsizer(bt.sizers.FixedSize, stake=1)

# Set the slippage (0.1%)
cerebro.broker.set_slippage_perc(0.001)

# Print the starting cash
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run the backtest
results = cerebro.run()

# Print the final portfolio value
print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Plot the results
cerebro.plot(style='candlestick')


Starting Portfolio Value: 10000.00
Ending Portfolio Value: 10000.00


[[<Figure size 640x480 with 57 Axes>]]

In [138]:
print(df.head())  # Check the first few rows of data
print(df.tail())  # Check the last few rows

                     open  high   low  close     volume
datetime                                               
2011-12-31 07:00:00  4.39  4.39  4.39   4.39   2.277904
2011-12-31 08:00:00  4.39  4.39  4.39   4.39  27.334852
2011-12-31 09:00:00  4.39  4.39  4.39   4.39  27.334852
2011-12-31 10:00:00  4.39  4.39  4.39   4.39  27.334852
2011-12-31 11:00:00  4.39  4.39  4.39   4.39  27.334852
                        open     high      low    close      volume
datetime                                                           
2021-03-04 04:00:00  49595.4  49847.4  49022.9  49050.0  353.736679
2021-03-04 05:00:00  49050.0  49740.0  49039.1  49684.9  188.363947
2021-03-04 06:00:00  49685.0  49905.7  49287.1  49411.2  191.054063
2021-03-04 07:00:00  49410.0  49840.7  49330.9  49696.5  155.019282
2021-03-04 08:00:00  49725.1  50750.0  49725.1  50235.2  248.692561


In [139]:
import backtrader as bt
import pandas as pd

# Load the data
df = pd.read_csv('/content/BTC1H.csv')

# Convert the datetime column to datetime format
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

# Create a Backtrader data feed from the DataFrame
data = bt.feeds.PandasData(dataname=df)

# Define the Moving Average Crossover Strategy
class MovingAverageCrossover(bt.Strategy):
    def __init__(self):
        # Define the moving averages using the 'close' price
        self.short_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=10)
        self.long_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=30)

        # Define crossover indicator
        self.cross_over = bt.indicators.CrossOver(self.short_ma, self.long_ma)

    def next(self):
      print('Short MA:', self.short_ma[0], 'Long MA:', self.long_ma[0], 'CrossOver:', self.cross_over[0])

      if self.cross_over > 0:
          if not self.position:
              self.buy()
              print('Buy signal at', self.data.datetime.datetime(), 'Price:', self.data.close[0])

      elif self.cross_over < 0:
          if self.position:
              self.sell()
              print('Sell signal at', self.data.datetime.datetime(), 'Price:', self.data.close[0])

      # For debugging purposes, you can plot the short and long MA
      self.plot()


# Initialize the backtest engine (Cerebro)
cerebro = bt.Cerebro()

# Add the data to the engine
cerebro.adddata(data)

# Set the initial cash (capital) for the backtest
cerebro.broker.set_cash(10000)

# Set the commission for each trade (0.1%)
cerebro.broker.setcommission(commission=0.001)

# Set the position size (1 BTC per trade)
cerebro.addsizer(bt.sizers.FixedSize, stake=1)

# Set the slippage (0.1%)
cerebro.broker.set_slippage_perc(0.001)

# Print the starting cash
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run the backtest
results = cerebro.run()

# Print the final portfolio value
print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Plot the results
cerebro.plot(style='candlestick')

Starting Portfolio Value: 10000.00
Ending Portfolio Value: 10000.00


[[<Figure size 640x480 with 61 Axes>]]